# Agent Executor From Scratch

In this notebook we will go over how to build a basic agent executor from
scratch.

![diagram](./img/agent-executor-diagram.png)

## Setup¶

First we need to install the packages required

```bash
yarn add langchain @langchain/openai @langchain/langgraph
```

Next, we need to set API keys for OpenAI (the LLM we will use) and Tavily (the
search tool we will use)

```bash
export OPENAI_API_KEY=
export TAVILY_API_KEY=
```

Optionally, we can set API key for
[LangSmith tracing](https://smith.langchain.com/), which will give us
best-in-class observability.

```bash
export LANGCHAIN_TRACING_V2=true
export LANGCHAIN_API_KEY=
```

## Create the LangChain agent

First, we will create the LangChain agent. For more information on LangChain
agents, see [this documentation](https://js.langchain.com/docs/modules/agents/).

In [2]:
// Deno.env.set("OPENAI_API_KEY", "YOUR_API_KEY")
// Deno.env.set("TAVILY_API_KEY", "YOUR_API_KEY")

In [1]:
import { pull } from "langchain/hub";
import { createOpenAIFunctionsAgent } from "langchain/agents";
import { ChatOpenAI } from "@langchain/openai";
import { TavilySearchResults } from "@langchain/community/tools/tavily_search";
import { ChatPromptTemplate } from "@langchain/core/prompts";

const tools = [new TavilySearchResults({ maxResults: 1 })];

// Get the prompt to use - you can modify this!
const prompt = await pull<ChatPromptTemplate>(
  "hwchase17/openai-functions-agent",
);

// Choose the LLM that will drive the agent
const llm = new ChatOpenAI({
  modelName: "gpt-4o",
  temperature: 0,
});

// Construct the OpenAI Functions agent
const agentRunnable = await createOpenAIFunctionsAgent({
  llm,
  tools,
  prompt,
});

## Define the graph schema

We now define the graph state. The state for the traditional LangChain agent has
a few attributes:

1. `input`: This is the input string representing the main ask from the user,
   passed in as input.
2. `steps`: This is list of actions and corresponding observations that the
   agent takes over time. This is updated each iteration of the agent.
3. `agentOutcome`: This is the response from the agent, either an AgentAction or
   AgentFinish. The AgentExecutor should finish when this is an AgentFinish,
   otherwise it should call the requested tools.

In [11]:
import { Annotation } from "@langchain/langgraph";
import { AgentAction, AgentFinish, AgentStep } from "@langchain/core/agents";

const AgentState = Annotation.Root({
  input: Annotation<string>,
  steps: Annotation<AgentStep[]>({
    reducer: (x, y) => x.concat(y),
    default: () => [],
  }),
  agentOutcome: Annotation<AgentAction | AgentFinish>,
});

## Define the nodes

We now need to define a few different nodes in our graph. In `langgraph`, a node
can be either a function or a
[runnable](https://js.langchain.com/docs/expression_language/). There are two
main nodes we need for this:

1. The agent: responsible for deciding what (if any) actions to take.
2. A function to invoke tools: if the agent decides to take an action, this node
   will then execute that action.

We will also need to define some edges. Some of these edges may be conditional.
The reason they are conditional is that based on the output of a node, one of
several paths may be taken. The path that is taken is not known until that node
is run (the LLM decides).

1. Conditional Edge: after the agent is called, we should either: a. If the
   agent said to take an action, then the function to invoke tools should be
   called b. If the agent said that it was finished, then it should finish
2. Normal Edge: after the tools are invoked, it should always go back to the
   agent to decide what to do next

Let's define the nodes, as well as a function to decide how what conditional
edge to take.

In [13]:
import { BaseMessage } from "@langchain/core/messages";
import { ToolExecutor } from "@langchain/langgraph/prebuilt";
import type { RunnableConfig } from "@langchain/core/runnables";

interface AgentStateBase {
  agentOutcome?: AgentAction | AgentFinish;
  steps: Array<AgentStep>;
}

interface AgentState extends AgentStateBase {
  input: string;
  chatHistory?: BaseMessage[];
}

const toolExecutor = new ToolExecutor({
  tools,
});

// Define logic that will be used to determine which conditional edge to go down
const shouldContinue = (data: AgentState) => {
  if (data.agentOutcome && "returnValues" in data.agentOutcome) {
    return "end";
  }
  return "continue";
};

const runAgent = async (data: AgentState, config?: RunnableConfig) => {
  const agentOutcome = await agentRunnable.invoke(data, config);
  return {
    agentOutcome,
  };
};

const executeTools = async (data: AgentState, config?: RunnableConfig) => {
  const agentAction = data.agentOutcome;
  if (!agentAction || "returnValues" in agentAction) {
    throw new Error("Agent has not been run yet");
  }
  const output = await toolExecutor.invoke(agentAction, config);
  return {
    steps: [{ action: agentAction, observation: JSON.stringify(output) }],
  };
};

## Define the graph

We can now put it all together and define the graph!

In [14]:
import { RunnableLambda } from "@langchain/core/runnables";
import { END, START, StateGraph } from "@langchain/langgraph";

// Define a new graph
const workflow = new StateGraph(AgentState)
  // Define the two nodes we will cycle between
  .addNode("agent", new RunnableLambda({ func: runAgent }))
  .addNode("action", new RunnableLambda({ func: executeTools }))
  // Set the entrypoint as `agent`
  // This means that this node is the first one called
  .addEdge(START, "agent")
  // We now add a conditional edge
  .addConditionalEdges(
    // First, we define the start node. We use `agent`.
    // This means these are the edges taken after the `agent` node is called.
    "agent",
    // Next, we pass in the function that will determine which node is called next.
    shouldContinue,
    // Finally we pass in a mapping.
    // The keys are strings, and the values are other nodes.
    // END is a special node marking that the graph should finish.
    // What will happen is we will call `should_continue`, and then the output of that
    // will be matched against the keys in this mapping.
    // Based on which one it matches, that node will then be called.
    {
      // If `tools`, then we call the tool node.
      continue: "action",
      // Otherwise we finish.
      end: END,
    },
  )
  // We now add a normal edge from `tools` to `agent`.
  // This means that after `tools` is called, `agent` node is called next.
  .addEdge("action", "agent");

const app = workflow.compile();

In [19]:
const inputs = { input: "what is the weather in sf" };
for await (const s of await app.stream(inputs)) {
  console.dir(s, { depth: 5 });
  console.log("----\n");
}

{
  agent: {
    agentOutcome: {
      tool: 'tavily_search_results_json',
      toolInput: { input: 'current weather in San Francisco' },
      log: 'Invoking "tavily_search_results_json" with {"input":"current weather in San Francisco"}\n',
      messageLog: [
        AIMessage {
          lc_serializable: true,
          lc_kwargs: {
            content: '',
            tool_calls: [],
            invalid_tool_calls: [],
            additional_kwargs: [Object],
            response_metadata: [Object],
            id: 'chatcmpl-9y1NwDZx83JESI1fojFvUue4sk5YS'
          },
          lc_namespace: [ 'langchain_core', 'messages' ],
          content: '',
          name: undefined,
          additional_kwargs: { function_call: [Object], tool_calls: undefined },
          response_metadata: {
            tokenUsage: [Object],
            finish_reason: 'function_call',
            system_fingerprint: 'fp_3aa7262c27'
          },
          id: 'chatcmpl-9y1NwDZx83JESI1fojFvUue4sk5YS',
     